In [1]:
import pickle
filename = 'finalized_model.pkl'

with open(filename, 'rb') as pickle_file:
    content = pickle.load(pickle_file)

In [2]:
content.feature_importances_

array([3.77990929e-03, 1.70734688e-02, 2.60327190e-02, 8.34729518e-03,
       2.12080591e-02, 1.92033818e-03, 1.01796582e-01, 8.54020568e-05,
       6.41102781e-05, 3.13296529e-01, 1.39790294e-03, 2.55233988e-01,
       2.49763697e-01])

### Set up
pip install azureml-sdk
pip install azureml-core

#### Reference:
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-existing-model

#### Where to deploy: 
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

## Connect to Azure ML Workspace

In [2]:

from azureml.core import Workspace
ws = Workspace.get(name='AzureMLWorkspaceName',
               subscription_id='XXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX',
               resource_group='ResourceGroupName'
               )

## Register existing ML model (in pickle format)

In [4]:
from azureml.core.model import Model

model = Model.register(model_path="finalized_model.pkl",
                          model_name="finalized_model",
                          description="Test register model from external",
                          workspace=ws)

Registering model finalized_model


## This will put in score_pbi2.py file

In [27]:
import os
import pickle
import json
import numpy as np
import pandas as pd

import sklearn
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


# Called when the deployed service starts
def init():
    global model

    # Get the path where the deployed model can be found.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'))
    # load models
    with open(model_path + '/finalized_model.pkl', 'rb') as handle:
        model = pickle.load(handle)



input_sample = pd.DataFrame(data=[{
    "alcohol": 13.58,
    "malic_acid": 2.58,
    "ash": 2.69,
    "alcalinity_of_ash": 24.5,
    "magnesium": 105,
    "total_phenols": 1.55,
    "flavanoids": 0.84,
    "nonflavanoid_phenols": 0.39,
    "proanthocyanins": 1.54,
    "color_intensity": 8.66,
    "hue": 0.74 ,
    "od280/od315_of_diluted_wines":1.8,
    "proline":750

}])

# This is an integer type sample. Use the data type that reflects the expected result.
output_sample = np.array([0])

# To indicate that we support a variable length of data input,
# set enforce_shape=False
@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        print("input_data....")
        print(data.columns)
        print(type(data))
        result = model.predict(data)
        print("result.....")
        print(result)
    # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

## Define inference configuration

In [70]:
!pip install inference_schema

  Created wheel for wrapt: filename=wrapt-1.11.1-py3-none-any.whl size=19547 sha256=dc65eb1953dc7235c75f601e8539d88840221f5bbec55ff8efd3217d627ee180
  Stored in directory: c:\users\wipadac\appdata\local\pip\cache\wheels\90\d4\83\58efda72eb47567053254faec24fe048dced315a0c3f11e8f8
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2


In [5]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="pbiendpoint")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
#conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("keras")
#conda_dep.add_pip_package("gensim")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

for pip_package in ["inference_schema"]:
    myenv.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script="score_pbi2.py",
                                   environment=myenv)

In [8]:
# Test on local deployment 
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration()
model = Model(ws, name='finalized_model')
service = Model.deploy(ws, 'pbiendpoint2', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

# If pass then deploy to ML Service

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 7f82caa54ce5444dbca309d665b6325b.azurecr.io
Logging into Docker registry 7f82caa54ce5444dbca309d665b6325b.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 7f82caa54ce5444dbca309d665b6325b.azurecr.io/azureml/azureml_f5c963dc6f1e8736d24918005d374e59
 ---> 4cbd8330689f
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 62a8e0d1aede
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjQ5MTRmMjYyLWJkYTgtNDZjYy1hOWRiLWM5Y2JkNjk0YjExNyIsInJlc291cmNlR3JvdXBOYW1lIjoiZGF0YWJyaWNrIiwiYWNjb3VudE5hbWUiOiJmb3JlZGVwbG95bW9kZWwiLCJ3b3Jrc3BhY2VJZCI6IjdmODJjYWE1LTRjZTUtNDQ0ZC1iY2EzLTA5ZDY2NWI2MzI1YiJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 42e5357a1b8a
 ---> cbc8767e4475
Step 4/5 : RUN mv '/var/azureml-app/tmpi_h73kz8.py' /var/azureml-app/main.py
 ---> Running in faef27cf54d

In [23]:
from azureml.core.webservice import AciWebservice, Webservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "pbiendpoint", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI: http://9547dae0-e3cc-4638-9d4e-e37fe26b3826.southeastasia.azurecontainer.io/score


In [1]:
#Test Calling Service 
X_df = pd.read_csv('test_scoring_nopredict.csv')

import json

input_payload = json.dumps({
    'data': X_df[0:2].values.tolist()
 # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

NameError: name 'X_df' is not defined